# Lab 1.2: Conversational RAG - Mortgage Policy Assistant

In this lab, we will build a **Banking Policy Assistant** for Wells Fargo. This assistant allows loan officers to query internal mortgage guidelines (e.g., Conforming vs. Jumbo loans) and ask follow-up questions while maintaining conversational context.

## Use Case
A loan officer needs to quickly check credit score requirements for different loan types without searching through a 200-page PDF manual. They might ask:
1. "What is the min credit score for a Jumbo loan?"
2. "How does that compare to FHA?" (Contextual follow-up)

## Key Concepts
1. **History Aware Retriever**: Rephrases the user's latest query using the chat history so it makes sense as a standalone query to the vector store.
2. **Chat History**: Maintaining line of conversation.

In [1]:
# 1. Install Dependencies
print("Installing dependencies...")
%pip install -qU langchain langchain-groq langchain-community langchain-huggingface chromadb sentence-transformers
print("Dependencies installed.")

Installing dependencies...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.2/490.2 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [2]:
# 2. Setup API Keys
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

Enter your Groq API Key: ··········


In [4]:
banking_policy_text = """
Wells Fargo Mortgage Lending Guidelines - Internal Policy Use Only

1. CONFORMING LOANS
   - Definition: Loans that meet the purchase limits set by Fannie Mae and Freddie Mac.
   - Max Loan Amount: $766,550 for single-family homes (as of 2024).
   - Minimum Credit Score: 620.
   - Down Payment: Minimum 3% for first-time homebuyers, otherwise 5%.
   - DTI Ratio: Maximum 50% with compensating factors.
   - Reserves: Not always required, depending on DU/LP findings.

2. JUMBO LOANS
   - Definition: Loans exceeding the conforming loan limits.
   - Minimum Credit Score: 700 for LTV up to 80%; 720 for LTV up to 90%.
   - Down Payment: Minimum 10% required.
   - DTI Ratio: Strictly capped at 43%.
   - Reserves: 6 months of PITI required.
   - Appraisal: Two full appraisals required for loan amounts > $1.5M.

3. FHA LOANS
   - Definition: Government-backed loans insured by the Federal Housing Administration.
   - Minimum Credit Score: 580 for 3.5% down payment; 500-579 for 10% down payment.
   - DTI Ratio: Up to 57% allowed in some cases.
   - MIP (Mortgage Insurance Premium): Upfront 1.75% + Annual MIP required for the life of the loan if LTV > 90%.
   - Property Requirements: Must meet FHA safety guidelines (no peeling paint, safety handrails required).

4. VA LOANS
   - Definition: Loans for eligible veterans and service members.
   - Down Payment: 0% required.
   - PMI: No private mortgage insurance required.
   - Funding Fee: Required unless the veteran has a service-connected disability.
   - DTI Ratio: No strict cap, but residual income analysis is key.

5. GENERAL UNDERWRITING REQUIREMENTS
   - Income: 2 years of consistent employment history required. Self-employed borrowers need 2 years of tax returns.
   - Assets: Large deposits must be sourced and explained.
   - Bankruptcy:
     - Chapter 7: 4-year waiting period for Conventional, 2 years for FHA/VA.
     - Chapter 13: 2-year waiting period after discharge for Conventional, 1 year of payout period for FHA/VA.
"""

In [5]:
# 3. Setup Vector Store (Mortgage Policy Data)
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

# Load Internal Policy Document
print("Loading mortgage policy document...")
docs = [Document(page_content=banking_policy_text, metadata={"source": "internal_policy_doc"})]
print(f"Loaded {len(docs)} document(s).")

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(docs)
print(f"Split into {len(splits)} chunks.")

# Embed & Store
print("Creating vector store...")
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever()
print("Vector store created.")

Loading mortgage policy document...
Loaded 1 document(s).
Split into 5 chunks.
Creating vector store...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector store created.


## 4. History Aware Retriever
We need a chain that takes the `chat_history` and the `input` and generates a search query.

In [6]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Initialize LLM
llm = ChatGroq(
    model="qwen/qwen3-32b",
    temperature=0,
    reasoning_format="parsed"
)

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Chain to rephrase question
contextualize_q_chain = contextualize_q_prompt | llm | StrOutputParser()
print("Contextualization chain created.")

Contextualization chain created.


In [7]:
# Let's test the contextualization to understand what it does
from langchain_core.messages import HumanMessage, AIMessage

print("--- Testing Query Contextualization ---")
# Mock history: user asked about Jumbo loans, AI answered.
sample_history = [
    HumanMessage(content="What is the min credit score for a Jumbo Loan?"),
    AIMessage(content="The minimum credit score is 700.")
]
# User asks a follow-up specific to the context (referring to 'that')
sample_input = "How does that compare to FHA?"
print(f"Chat History: {len(sample_history)} messages")
print(f"User Follow-up: {sample_input}")

rephrased_query = contextualize_q_chain.invoke({"chat_history": sample_history, "input": sample_input})
print(f"Rephrased Query (for Vector Store): {rephrased_query}")
print("---------------------------------------")

--- Testing Query Contextualization ---
Chat History: 2 messages
User Follow-up: How does that compare to FHA?
Rephrased Query (for Vector Store): What is the minimum credit score for an FHA loan compared to a Jumbo Loan?
---------------------------------------


## 5. QA Chain with History
Now we create the final chain that uses the retrieved documents to answer.

In [8]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_system_prompt = (
    "You are a specialized Mortgage Policy Assistant for Wells Fargo. "
    "Use the following pieces of retrieved policy context to answer "
    "the loan officer's question. If you don't know the answer, say that you "
    "cannot find it in the policy. Keep answers professional and concise."
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input.get("input")

rag_chain = (
    RunnablePassthrough.assign(
        context=contextualized_question | retriever | format_docs
    )
    | qa_prompt
    | llm
    | StrOutputParser()
)
print("Full RAG chain created.")

Full RAG chain created.


## 6. Testing the Chat
We can now manage specific chat sessions.

In [9]:
chat_history = []

# First Question: Specific Policy Query
user_input = "What is the minimum credit score for a Jumbo Loan?"

print(f"User: {user_input}")
# rag_chain returns string now
answer = rag_chain.invoke({"input": user_input, "chat_history": chat_history})
print(f"AI: {answer}")

# Update History
chat_history.extend([HumanMessage(content=user_input), AIMessage(content=answer)])

# Second Question (Follow-up): Contextual Comparison
print("\n--- Follow up ---")
user_input = "How does that compare to FHA loans?"
print(f"User: {user_input}")

answer = rag_chain.invoke({"input": user_input, "chat_history": chat_history})
print(f"AI: {answer}")

chat_history.extend([HumanMessage(content=user_input), AIMessage(content=answer)])

User: What is the minimum credit score for a Jumbo Loan?
AI: The minimum credit score for a Jumbo Loan is **700 for a loan-to-value (LTV) ratio up to 80%**, and **720 for an LTV ratio up to 90%**. These requirements are outlined in the Wells Fargo Jumbo Loan policy.

--- Follow up ---
User: How does that compare to FHA loans?
AI: The minimum credit score for **FHA loans** is significantly lower than for Jumbo Loans. For FHA loans:  
- **580** for a 3.5% down payment.  
- **500–579** for a 10% down payment.  

This contrasts with Jumbo Loans, which require **700 (LTV ≤80%)** or **720 (LTV ≤90%)**. FHA loans are designed to assist borrowers with lower credit scores, while Jumbo Loans have stricter credit requirements due to their higher risk profile.
